# Neural Machine Translation

General Reference: https://github.com/nyu-dl/NLP_DL_Lecture_Note/blob/master/lecture_note.pdf <br>
Original Notebook: https://github.com/nyu-dl/AMMI-2019-NLP-Part2

### Install packages

In [ ]:
!pip install torch
!pip install subword-nmt
!pip install sacremoses
!pip install googletrans==3.1.0a0
!pip install pandas
!pip install sacrebleu
!pip install matplotlib
!pip install bertviz

### Set up Google Translate API for Comparison

https://github.com/ssut/py-googletrans

In [1]:
from googletrans import Translator
google_translator = Translator()

### Python imports

In [2]:
"""
To run this notebook in Google Colab, you need to the following first:
1. Go to "Runtime / Change runtime type", then select "GPU" in the "Hardware accelerator" drop-down list
2. Open this link: https://drive.google.com/drive/folders/1E07YaKths98YpoBCH2PjdtTPqOXgfdZB?usp=sharing
3. Then go to "Shared with me" in your Google Drive, right-click the "ALPS2022-NMT" folder
and select "Add shortcut to Drive"
"""

colab = False   # set to False to run locally and not from Google Colab

if colab:
    # Download the python files from the ALPS Github
    !wget https://raw.githubusercontent.com/naverlabseurope/ALPS2021-MT-LAB/ALPS2022/data.py
    !wget https://raw.githubusercontent.com/naverlabseurope/ALPS2021-MT-LAB/ALPS2022/models.py
    # Mount your Google Drive, which should contain a link to "ALPS2022-NMT"
    from google.colab import drive
    drive.flush_and_unmount()
    drive.mount('/content/drive')
    root_dir = '/content/drive/MyDrive/ALPS2022-NMT'   # TODO: upload new data and pre-trained models
    !ls {root_dir}/*
else:
    # Download the datasets and pre-trained models
    # Modify this script to download data in other language pairs than EN-FR
    !scripts/download-data.sh
    root_dir = '.'

import os
import sys
import data
import models
import numpy as np
import torch
import time
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from subword_nmt.apply_bpe import BPE
%matplotlib inline

## The Dataset

We will work with a small English to French dataset from https://www.manythings.org/anki/. It contains translations of short and simple sentences aimed at foreign language learners (from the [Tatoeba collaborative database](https://tatoeba.org/en/)). Of course, models trained on this data will not perform well on longer, more sophisticated sentences. They also won't be very robust to domain shift and input noise. To train stronger models, some larger datasets can be downloaded from https://www.statmt.org/wmt21/ or https://opus.nlpl.eu/.

In [3]:
source_lang, target_lang = 'en', 'fr'
data_dir = os.path.join(root_dir, 'data')
model_dir = os.path.join(root_dir, 'models', f'{source_lang}-{target_lang}')
!head -5 {data_dir}/train.en-fr.en

is it cancer ?
it took half an hour .
how many caps do you own ?
you should read many books when you are young .
he is a student at harvard .


## Load and preprocess the data

1. Load the BPE model
2. Load the parallel corpora for this language pair (train, valid and test). `load_data` will load a corpus and tokenize it with the BPE model with the `preprocess` function.
3. Create (or load) dictionaries that map BPE tokens to token IDs (`data.load_or_create_dictionary` function)
4. Binarize the data: map source and target text sequences to sequences of IDs, and sort the training set by length (`data.binarize` function)
5. Create batches (`data.BatchIterator` class): group multiple sequence pairs of similar length together, pad them to the maximum length and create numpy arrays that can be used to train our models

In [4]:
def reset_seed(seed=1234):
    np.random.seed(seed)
    torch.manual_seed(seed)

#### 1. Load the BPE model (multilingual BPE model, works with French, German and English)

In [5]:
bpe_path = os.path.join(data_dir, 'bpecodes.de-en-fr')

with open(bpe_path) as bpe_codes:
    bpe_model = BPE(bpe_codes)

def preprocess(line, is_source=True, source_lang=None, target_lang=None):
    return bpe_model.segment(line.lower())

def postprocess(line):
    return line.replace('@@ ', '')

def load_data(source_lang, target_lang, split='train', max_size=None):
    # max_size: max number of sentence pairs in the training corpus (None = all)
    path = os.path.join(data_dir, f'{split}.{source_lang}-{target_lang}')
    return data.load_dataset(path, source_lang, target_lang, preprocess=preprocess, max_size=max_size)   # set max_size to 10000 for fast debugging

#### 2. Load and preprocess the parallel corpora (these are pandas DataFrames)

In [6]:
train_data = load_data(source_lang, target_lang, 'train', max_size=None)   # set max_size to 10000 for fast debugging
valid_data = load_data(source_lang, target_lang, 'valid')
test_data = load_data(source_lang, target_lang, 'test')
print(train_data.iloc[:5])

                                       source_data  \
0                                   is it cancer ?   
1                           it took half an hour .   
2                       how many caps do you own ?   
3  you should read many books when you are young .   
4                     he is a student at harvard .   

                                         target_data  \
0                          s' agit-il d' un cancer ?   
1                         ça a pris une demi-heure .   
2                combien de casquettes possèdes-tu ?   
3  on devrait lire beaucoup de livres quand on es...   
4                        il est étudiant à harvard .   

                                    source_tokenized  \
0                                [is, it, cancer, ?]   
1                      [it, took, half, an, hour, .]   
2             [how, many, cap@@, s, do, you, own, ?]   
3  [you, should, read, many, books, when, you, ar...   
4           [he, is, a, student, at, harv@@, ard, .]   

 

#### 3. Load or create the dictionaries

In [7]:
source_dict_path = os.path.join(model_dir, f'dict.{source_lang}.txt')
target_dict_path = os.path.join(model_dir, f'dict.{target_lang}.txt')

source_dict = data.load_or_create_dictionary(
    source_dict_path,
    train_data['source_tokenized'],
    minimum_count=10,
    reset=False,    # set reset to True if you're changing the data or the preprocessing
)
print(source_dict.words[:100])

target_dict = data.load_or_create_dictionary(
    target_dict_path,
    train_data['target_tokenized'],
    minimum_count=10,
    reset=False,
)
print(target_dict.words[:100])

['<sos>', '<eos>', '<unk>', '<pad>', '.', 'i', 'the', 'you', 'to', '?', "'t", 'a', 'in', 'he', 'is', 'that', 'tom', 'it', "'m", "'s", 'think', 'not', 'of', 'have', ',', 'don', 'do', 'your', 'this', 'we', 'me', 'for', 'on', 'was', 'his', 'what', 'want', 'be', 'are', 'can', "'re", 'she', 'my', 'if', 'about', 'like', 'at', 'him', 'go', 'how', "'ll", 'here', 'with', 'there', 'time', 'get', 'did', 'has', 'will', 'didn', 'know', 'up', 'her', 'they', 'all', 'e@@', 'please', 'y', 'so', 'were', 'going', 'take', 'al', "'ve", 'who', 'had', 'when', 'no', 'now', 'one', 'something', 'been', 'need', 'ed', 'very', 'never', 'an', 'too', 'tell', 'and', 'why', 'let', 'see', 'got', 'room', 'come', 'should', 'by', 'where', 'right']
['<sos>', '<eos>', '<unk>', '<pad>', '.', 'je', 'pas', 'vous', 'de', '?', 'ne', 'en', 'est', 'la', 'le', 'les', "l'", 'un', 'il', 'a', 'à', 'que', 'tom', ',', "n'", "c'", 'ce', 'tu', 'ai', 'pour', 'nous', "j'", "m'", '!', 'des', 'me', 'ça', 'si', "d'", "s'", 'elle', 'était', 'fa

In [8]:
print('source vocab size:', len(source_dict))
print('target vocab size:', len(target_dict))

source vocab size: 3134
target vocab size: 3607


#### 4. Use the dictionaries to map tokens to indices. The training set is also sorted by length for more efficient batching.

In [9]:
data.binarize(train_data, source_dict, target_dict, sort=True)
data.binarize(valid_data, source_dict, target_dict, sort=False)
data.binarize(test_data, source_dict, target_dict, sort=False)
print(train_data.iloc[:5])

        source_data      target_data  source_tokenized    target_tokenized  \
354        enough !          assez !       [enough, !]          [assez, !]   
1105      finally !          enfin !      [finally, !]          [enfin, !]   
4365         fire !            feu !         [fire, !]            [feu, !]   
7206  immediately !  immédiatement !  [immediately, !]  [immédiatement, !]   
9258         sing !         chante !         [sing, !]         [chante, !]   

          source_bin  source_len     target_bin  target_len  
354    [296, 261, 1]           3   [254, 33, 1]           3  
1105  [1133, 261, 1]           3  [1998, 33, 1]           3  
4365   [650, 261, 1]           3   [644, 33, 1]           3  
7206  [1144, 261, 1]           3   [874, 33, 1]           3  
9258   [417, 261, 1]           3  [2551, 33, 1]           3  


#### Data statistics:

In [ ]:
print('train_size={}, valid_size={}, test_size={}, min_len={}, max_len={}'.format(
    len(train_data),
    len(valid_data),
    len(test_data),
    train_data['source_len'].min(),
    train_data['source_len'].max(),
))

print('Train source length distribution:')
print(train_data['source_len'].quantile([0.5, 0.75, 0.9, 0.95, 0.99, 0.999, 0.9999]))

#### 5. Build batches. The training batches are automatically shuffled before each epoch

In [ ]:
max_len = 30       # maximum 30 tokens per sentence (longer sequences will be truncated)
batch_size = 512   # maximum 512 tokens per batch (decrease if you get OOM errors, increase to speed up training)

reset_seed()

train_iterator = data.BatchIterator(train_data, source_lang, target_lang, batch_size=batch_size, max_len=max_len, shuffle=True)
valid_iterator = data.BatchIterator(valid_data, source_lang, target_lang, batch_size=batch_size, max_len=max_len, shuffle=False)
test_iterator = data.BatchIterator(test_data, source_lang, target_lang, batch_size=batch_size, max_len=max_len, shuffle=False)

#### Example of training batch:

In [ ]:
print(next(iter(train_iterator)))

The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A `Sequence to Sequence network <http://arxiv.org/abs/1409.3215>`__, or
seq2seq network, or `Encoder-Decoder
network <https://arxiv.org/pdf/1406.1078v3.pdf>`__, is a model
consisting of usually two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence. Essentially, all we need is some mechanism to read the source sentence and create an encoding and some mechanism to read the encoding and decode it to the target language. 

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

Consider the sentence "I am not the
black cat" → "Je ne suis pas le chat noir". Most of the words in the input sentence have a direct
translation in the output sentence, but are in slightly different
orders, e.g. "chat noir" and "black cat". Because of the "ne/pas"
construction there is also one more word in the input sentence. It would
be difficult to produce a correct translation directly from the sequence
of input words.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the meaning of the input sequence into a single
vector — a single point in some N dimensional space of sentences.


The Encoder
-----------

The encoder is anything which takes in a sentence and gives us a representation for the sentence. 

The encoder of a seq2seq network can be a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.

However, we will start with a simpler Bag-of-Words encoder and then move on to more complex encoders.

### Bag-of-Words Encoder

In [ ]:
bow_encoder = models.BagOfWords(
    input_size=len(source_dict),
    hidden_size=512,
    num_layers=1,
    dropout=0.0,
    reduce='sum',
)

In [ ]:
print(bow_encoder)

The Decoder
--------------------

The decoder is another network that takes the encoder output vector(s) and outputs a sequence of words to create the translation.

### Decoder without Attention

In the simplest seq2seq decoder we use only the last output of the encoder. This last output is sometimes called the context vector as it encodes context from the entire sequence. This context vector can be used as the initial hidden state for an RNN decoder.

At every step of decoding, the decoder is given an input token and hidden state. The initial input token is the start-of-string <SOS> token, and the first hidden state is the context vector (the encoder's last hidden state).

In [ ]:
bow_decoder = models.RNN_Decoder(
    output_size=len(target_dict),
    hidden_size=512,
    num_layers=1,
    dropout=0.0,
)

In [ ]:
print(bow_decoder)

In [ ]:
bow_model = models.EncoderDecoder(
    bow_encoder,
    bow_decoder,
    lr=0.001,
    use_cuda=True,
    target_dict=target_dict,
)

### Training code

In [ ]:
def train_model(
        train_iterator,
        valid_iterators,
        model,
        checkpoint_path,
        epochs=10,
        validation_frequency=1
    ):
    """
    train_iterator: instance of data.BatchIterator or data.MultiBatchIterator
    valid_iterators: list of data.BatchIterator
    model: instance of models.EncoderDecoder
    checkpoint_path: path of the model checkpoint
    epochs: iterate this many times over train_iterator
    validation_frequency: validate the model every N epochs
    """
    epochs += model.epoch

    reset_seed()

    best_score = -1
    for epoch in range(model.epoch + 1, epochs + 1):

        start = time.time()
        running_loss = 0

        print(f'Epoch [{epoch}/{epochs}]')

        # Iterate over training batches for one epoch
        with tqdm(enumerate(train_iterator), total=len(train_iterator)) as t:

            for i, batch in t:
                running_loss += model.train_step(batch)
                t.postfix = f' loss={running_loss / (i + 1):.3f}'

        # Average training loss for this epoch
        epoch_loss = running_loss / len(train_iterator)

        print(f'loss={epoch_loss:.3f}, time={time.time() - start:.2f}')
        sys.stdout.flush()

        # Evaluate and save the model
        if epoch % validation_frequency == 0:
            scores = []
            
            # Compute chrF over all validation sets
            for valid_iterator in valid_iterators:
                src, tgt = valid_iterator.source_lang, valid_iterator.target_lang
                translation_output = model.translate(valid_iterator, postprocess)
                score = translation_output.score
                output = translation_output.output

                with open(os.path.join(model_dir, f'valid.{src}-{tgt}.{epoch}.out'), 'w') as f:
                    f.writelines(line + '\n' for line in output)

                print(f'{src}-{tgt}: chrF={score}')
                sys.stdout.flush()
                scores.append(score)

            # Average the validation chrF scores
            score = round(sum(scores) / len(scores), 2)
            if len(scores) > 1:
                print(f'chrF={score}')

            # Update the model's learning rate based on current performance.
            # This scheduler divides the learning rate by 10 if chrF does not improve.
            model.scheduler_step(score)

            # Save a model checkpoint if it has the best validation chrF so far
            if score > best_score:
                best_score = score
                model.save(checkpoint_path)

        print('=' * 50)

    print(f'Training completed. Best chrF is {best_score}')

### Train a model with BOW Encoder and RNN Decoder (or load a pre-trained model)

In [ ]:
# Set this value to True to train your own model. By default, a pre-trained model will be loaded.
# Tip: you can set "epochs" to a small value (e.g., 2) and re-run this cell several times to continue training you model (`train_model` does not reset the model)
train_again = False

if train_again:
    checkpoint_path = os.path.join(model_dir, 'bow.pt')
else:
    checkpoint_path = os.path.join(model_dir, 'pretrained-bow.pt')

print('checkpoint path:', checkpoint_path)

if os.path.exists(checkpoint_path) and not train_again:
    bow_model.load(checkpoint_path)   # trained for 10 epochs
else:
    train_model(train_iterator, [valid_iterator], bow_model,
                epochs=2,
                checkpoint_path=checkpoint_path)

### Compute chrF on the test set

In [ ]:
print('chrF:', bow_model.translate(test_iterator, postprocess).score)

### Interact with the model

In [ ]:
def show_attention(input_sentence, output_words, attentions):
    # Plot an encoder-decoder attention matrix
    fig = plt.figure()
    ax = fig.add_subplot(111)
    fig.colorbar(ax.matshow(attentions, cmap='bone', aspect='auto'))
    xlabels = input_sentence.split() + [data.EOS_TOKEN]
    ylabels = output_words.split() + [data.EOS_TOKEN]
    ax.set_xticks(range(len(xlabels)))
    ax.set_xticklabels(xlabels, rotation=90)
    ax.set_yticks(range(len(ylabels)))
    ax.set_yticklabels(ylabels)
    plt.show()


def encode_as_batch(sentence, dictionary, source_lang, target_lang):
    # Create a batch from a single sentence
    sentence = f'{sentence} {data.EOS_TOKEN}'
    tensor = dictionary.txt2vec(sentence).unsqueeze(0)
    return {
        'source': tensor,
        'source_len': torch.from_numpy(np.array([tensor.shape[-1]])),
        'source_lang': source_lang,
        'target_lang': target_lang,
    }


def get_translation(model, sentence, dictionary, source_lang, target_lang, return_output=False):
    # Translate given sentence with given model. Also show translation outputs by Google Translate for comparison.
    print('Source:', sentence)
    sentence_tok = preprocess(sentence, is_source=True, source_lang=source_lang, target_lang=target_lang)
    print('Tokenized source:', sentence_tok)
    batch = encode_as_batch(sentence_tok, dictionary, source_lang, target_lang)
    prediction, attn_matrix, enc_self_attn = model.eval_step(batch)
    prediction = prediction[0]
    prediction_detok = postprocess(prediction)
    print('Prediction:', prediction)
    print('Detokenized prediction:', prediction_detok)

    print('Google Translate ({}->{}): {}'.format(
        source_lang,
        target_lang,
        google_translator.translate(sentence, src=source_lang, dest=target_lang).text,
    ))
    print('Google Translate on prediction ({}->{}): {}'.format(
        target_lang,
        source_lang,
        google_translator.translate(prediction_detok, src=target_lang, dest=source_lang).text,
    ))

    results = {
        'source': sentence,
        'source_tokens': sentence_tok.split() + ['<eos>'],
        'prediction_detok': prediction_detok,
        'prediction_tokens': prediction.split(),
    }

    if attn_matrix is not None:
        attn_matrix = attn_matrix[0].detach().cpu().numpy()
        results['attention_matrix'] = attn_matrix
        show_attention(sentence_tok, prediction, attn_matrix)
    
    if enc_self_attn is not None:
        results['encoder_self_attention_list'] = enc_self_attn
    
    if return_output:
        return results

In [ ]:
get_translation(bow_model, 'hello how are you ?', source_dict, source_lang, target_lang)

The biggest limitation of a Bag-of-Word encoder is that it is insensitive to word order: <br>
when shuffling the words in the previous sentence, you get the same output.

In [ ]:
get_translation(bow_model, 'are hello ? how you', source_dict, source_lang, target_lang)

In [ ]:
get_translation(bow_model, "she 's five years older than me .", source_dict, source_lang, target_lang)

## RNN Encoder + RNN Decoder

In [ ]:
rnn_encoder = models.RNN_Encoder(
    input_size=len(source_dict),
    hidden_size=512,
    num_layers=1,
    dropout=0.2,
)

In [ ]:
print(rnn_encoder)

In [ ]:
rnn_decoder = models.RNN_Decoder(
    output_size=len(target_dict),
    hidden_size=512,
    num_layers=1,
    dropout=0.2,
)

In [ ]:
print(rnn_decoder)

In [ ]:
rnn_model = models.EncoderDecoder(
    rnn_encoder,
    rnn_decoder,
    lr=0.001,
    use_cuda=True,
    target_dict=target_dict,
)

### Train a model with RNN Encoder and RNN Decoder (or load a pre-trained model)

In [ ]:
# Set this value to True to train your own model. By default, a pre-trained model will be loaded.
# Tip: you can set "epochs" to a small value (e.g., 2) and re-run this cell several times to continue training you model (`train_model` does not reset the model)
train_again = False

if train_again:
    checkpoint_path = os.path.join(model_dir, 'rnn.pt')
else:
    checkpoint_path = os.path.join(model_dir, 'pretrained-rnn.pt')

print('checkpoint path:', checkpoint_path)

if os.path.exists(checkpoint_path) and not train_again:
    rnn_model.load(checkpoint_path)   # trained for 10 epochs
else:
    train_model(train_iterator, [valid_iterator], rnn_model,
                epochs=2,
                checkpoint_path=checkpoint_path)

### Compute chrF on the test set

In [ ]:
print('chrF:', rnn_model.translate(test_iterator, postprocess).score)

### Interact with the model

In [ ]:
get_translation(rnn_model, 'hello how are you ?', source_dict, source_lang, target_lang)

Contrary to the BoW encoder, an RNN is sensitive to word ordering

In [ ]:
get_translation(rnn_model, 'are hello ? how you', source_dict, source_lang, target_lang)

In [ ]:
get_translation(rnn_model, "she 's five years older than me .", source_dict, source_lang, target_lang)

In [ ]:
get_translation(rnn_model, 'i know that the last thing you want to do is help me .', source_dict, source_lang, target_lang)

## RNN Encoder + RNN Decoder with Encoder-Decoder Attention

In [ ]:
rnn_attn_encoder = models.RNN_Encoder(
    input_size=len(source_dict),
    hidden_size=512,
    num_layers=1,
    dropout=0.0,
)

In [ ]:
 print(rnn_attn_encoder)

In [ ]:
rnn_attn_decoder = models.AttentionDecoder(
    output_size=len(target_dict),
    hidden_size=512,
    dropout=0.0,
)

In [ ]:
print(rnn_attn_decoder)

In [ ]:
rnn_attn_model = models.EncoderDecoder(
    rnn_attn_encoder,
    rnn_attn_decoder,
    lr=0.001,
    use_cuda=True,
    target_dict=target_dict,
)

### Train a model with RNN Encoder and RNN Decoder with attention (or load a pre-trained model)

In [ ]:
# Set this value to True to train your own model. By default, a pre-trained model will be loaded.
# Tip: you can set "epochs" to a small value (e.g., 2) and re-run this cell several times to continue training you model (`train_model` does not reset the model)
train_again = False

if train_again:
    checkpoint_path = os.path.join(model_dir, 'rnn-attn.pt')
else:
    checkpoint_path = os.path.join(model_dir, 'pretrained-rnn-attn.pt')

print('checkpoint path:', checkpoint_path)

if os.path.exists(checkpoint_path) and not train_again:
    rnn_attn_model.load(checkpoint_path)   # trained for 10 epochs
else:
    train_model(train_iterator, [valid_iterator], rnn_attn_model,
                epochs=2,
                checkpoint_path=checkpoint_path)

### Compute chrF on the test set

In [ ]:
print('chrF:', rnn_attn_model.translate(test_iterator, postprocess).score)

### Interact with the model and visualize attention matrices

In [ ]:
get_translation(rnn_attn_model, 'hello how are you ?', source_dict, source_lang, target_lang)

In [ ]:
get_translation(rnn_attn_model, "she 's five years older than me .", source_dict, source_lang, target_lang)

In [ ]:
get_translation(rnn_attn_model, 'i know that the last thing you want to do is help me .', source_dict, source_lang, target_lang)

## Transformer Model

[Transformer](https://arxiv.org/abs/1706.03762) is currently the state-of-the-art for Machine Translation. The encoder uses self-attention over the previous layers. The decoder combines self-attention and encoder-decoder attention.

In [ ]:
transformer_encoder = models.TransformerEncoder(
    input_size=len(source_dict),
    hidden_size=512,
    num_layers=1,
    dropout=0.0,
    heads=4,
)

In [ ]:
print(transformer_encoder)

In [ ]:
transformer_decoder = models.TransformerDecoder(
    output_size=len(target_dict),
    hidden_size=512,
    num_layers=1,
    heads=4,
    dropout=0.0,
)

In [ ]:
print(transformer_decoder)

In [ ]:
transformer_model = models.EncoderDecoder(
    transformer_encoder,
    transformer_decoder,
    lr=0.001,
    use_cuda=True,
    target_dict=target_dict,
)

### Train a Transformer model (or load a pre-trained model)

In [ ]:
# Set this value to True to train your own model. By default, a pre-trained model will be loaded.
# Tip: you can set "epochs" to a small value (e.g., 2) and re-run this cell several times to continue training you model (`train_model` does not reset the model)
train_again = False

if train_again:
    checkpoint_path = os.path.join(model_dir, 'transformer.pt')
else:
    checkpoint_path = os.path.join(model_dir, 'pretrained-transformer.pt')

print('checkpoint path:', checkpoint_path)

if os.path.exists(checkpoint_path) and not train_again:
    transformer_model.load(checkpoint_path)   # trained for 10 epochs
else:
    train_model(train_iterator, [valid_iterator], transformer_model,
                epochs=2,
                checkpoint_path=checkpoint_path)

### Compute chrF on the test set

In [ ]:
print('chrF:', transformer_model.translate(test_iterator, postprocess).score)

### Interact with the model

In [ ]:
from bertviz import head_view, model_view

In [ ]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/5.7.0/d3.min',
    jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

In [ ]:
def show_head_view(results):
    self_attention = results['encoder_self_attention_list']
    tokens = results['source_tokens']
    sentence_b_start = None
    head_view(self_attention, tokens, sentence_b_start)

def show_model_view(results):
    self_attention = results['encoder_self_attention_list']
    tokens = results['source_tokens']
    sentence_b_start = None
    model_view(self_attention, tokens, sentence_b_start)

In [ ]:
results = get_translation(transformer_model, 'hello how are you ?', source_dict, source_lang, target_lang, return_output=True)

In [ ]:
show_head_view(results)

In [ ]:
show_model_view(results)

In [ ]:
results = get_translation(transformer_model, "she 's five years older than me .", source_dict, source_lang, target_lang, return_output=True)

In [ ]:
results = get_translation(transformer_model, 'i know that the last thing you want to do is help me .', source_dict, source_lang, target_lang, return_output=True)

## Multilingual Transformer model

Load a pre-trained **de, fr <-> en** model. The same dictionary and embeddings are shared between all languages, and language codes (`<lang:de>`, `<lang:en>`, `<lang:fr>`) are prepended to each source sequence to identify the target language.

In [ ]:
multi_model_dir = os.path.join('models', 'de-en-fr')

multi_dict = data.load_or_create_dictionary(
    os.path.join(multi_model_dir, 'dict.txt'),
    dataset=None,
    minimum_count=10,
    reset=False,
)

encoder = models.TransformerEncoder(input_size=len(multi_dict), hidden_size=512, num_layers=1, heads=4)
decoder = models.TransformerDecoder(output_size=len(multi_dict), hidden_size=512, num_layers=1, heads=4)
decoder.embedding = encoder.embedding
multi_model = models.EncoderDecoder(encoder, decoder, lr=0.001, use_cuda=True, target_dict=multi_dict)

checkpoint_path = os.path.join(multi_model_dir, 'pretrained-transformer.pt')
multi_model.load(checkpoint_path)


### Multilingual evaluation

Modify the `preprocess` function to automatically prepend language codes to all source sequences (when calling `get_translation`, or `load_data`).

And load test sets in all language pairs.

In [ ]:
def preprocess(line, is_source=True, source_lang=None, target_lang=None):
    line = bpe_model.segment(line.lower())
    if is_source:
        line = f'<lang:{target_lang}> {line}'
    return line

test_iterators = []

for pair in 'en-fr', 'fr-en', 'en-de', 'de-en', 'de-fr', 'fr-de':
    src, tgt = pair.split('-')
    dataset = load_data(src, tgt, 'test')
    data.binarize(dataset, source_dict=multi_dict, target_dict=multi_dict, sort=False)
    iterator = data.BatchIterator(dataset, src, tgt, batch_size=512, max_len=30, shuffle=False)
    test_iterators.append(iterator)

In [ ]:
for test_iterator in test_iterators[:4]:
    print('chrF {}-{}: {}'.format(
        test_iterator.source_lang,
        test_iterator.target_lang,
        multi_model.translate(test_iterator, postprocess).score,
    ))

### Interact with the model

In [ ]:
get_translation(multi_model, "she 's five years older than me .", multi_dict, source_lang='en', target_lang='fr')

In [ ]:
get_translation(multi_model, 'sie ist fünf jahre älter als ich .', multi_dict, source_lang='de', target_lang='en')

### Zero-shot translation

In theory, the model can do **zero-shot** translation, i.e., translate between German and French even though it has never seen German-French sentence pairs during training.

In [ ]:
for test_iterator in test_iterators[4:]:
    print('chrF {}-{}: {}'.format(
        test_iterator.source_lang,
        test_iterator.target_lang,
        multi_model.translate(test_iterator, postprocess).score,
    ))

#### However, in practice zero-shot performance is very bad. Interact with the model to understand why.

In [ ]:
get_translation(multi_model, 'sie ist fünf jahre älter als ich .', multi_dict, 'de', 'fr')

In [ ]:
get_translation(multi_model, 'elle a cinq ans de plus que moi .', multi_dict, 'fr', 'de')

## Your Turn!

Choose one of these exercises, or both!

### Hyper-parameter tuning

Find the best hyper-parameters for Transformer **en-fr**. Share your best test chrF scores on Slack!

*Don't forget to reload the `preprocess` function at the start of the notebook*

- Hyper-parameters: `lr`, `batch_size`, `num_layers`, `hidden_size`, `dropout`, `heads`, etc.
- Other improvements: modify the learning rate scheduler and optimizer in `models.EncoderDecoder`; use different embedding size and hidden size, etc.

### Multilingual NMT

Train your own multilingual NMT model.

Tips:
- Create a multilingual dictionary by concatenating the tokenized data in all languages. Or simply re-use the dictionary of the pre-trained model (`multi_dict`).
- Use the same dictionary for the source and target sides, and share the embeddings between your encoder and decoder (do: `decoder.embedding = encoder.embedding`).
- Use `data.MultiBatchIterator(iterator_list)` to concatenate a list of training iterators (one for each language pair) into a single iterator, which is compatible with `train_model`.
- `train_model` can take a list of several validation iterators, which will let you validate your model on several language pairs.
- Improve your model's performance on **de-fr** and **fr-de** by including training data for these languages pairs (`data/train.de-fr.de` and `data/train.de-fr.fr`).

### Train bilingual or multilingual models on other language pairs

- Modify and re-run `./download-data.sh` to download data in new languages, preprocess this data and train BPE models